<h1>Credit Score Prediction</h1>

The goal of this project is to predict a credit score from 304 attributes.


Supervised machine learning method implementation step are:

<ol>
  <li>EDA</li>
  <li>feature engineering</li>
  <li>feature selection</li>
  <li>classification</li>
  <li>discussion</li>
</ol>

This notebook includes steps 2-5.

<b>About the data from the publisher</b><br>
We provide you with a data set in CSV format.<br>
The data set contains 8,000 train instances and 2000 test instance<br>
There are 304 input features, labeled x001 to x304.<br>
The target variable is labeled y.<br>

<b>Task: Create a model to predict the target variable y.</b><br>
A report - A Power point presentation<br>
Any custom code you used<br>
Instructions for me to run your model on a separate data set<br>

<b>Link to the data set at Kaggle:</b><br> https://www.kaggle.com/prasy46/credit-score-prediction

<b>Results</b><br>
With the selected features credit scores were predicted with RMSE_train of 1037.0 and RMSE_test of 1065.4. R2_train 0.9260 and R2_test 0.9246.

<h2>2. Feature Engineering</h2><br>

In [ ]:
# importing libraries
from platform import python_version
import warnings

# for working with arrays and dataframes
import numpy as np
import pandas as pd

# for plotting
import matplotlib.pyplot as plt #for plotting data
import seaborn as sns # for plotting data

# for quasi-constant values detection, validation and evaluation
import sklearn
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

# for classification
import xgboost
from xgboost import XGBRegressor

In [ ]:
# showing versions
print('Python version:',python_version())
print('NumPy version:',np.__version__)
print('Pandas version:',pd.__version__)
print('Sklearn version:',sklearn.__version__)
print('XGBoost version:',xgboost.__version__)

Python version: 3.7.14
NumPy version: 1.21.6
Pandas version: 1.3.5
Sklearn version: 1.0.2
XGBoost version: 0.90


In [ ]:
# settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [ ]:
# importing the train set
df_train = pd.read_csv("CreditScore_train.csv")
print(f"{df_train.shape[1]} columns, {df_train.shape[0]} rows")
# checking duplicated rows
print(f"{df_train.duplicated().sum()} duplicated rows")
df_train.head()

305 columns, 80000 rows
0 duplicated rows


,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x019,x020,x021,x022,x023,x024,x025,x026,x027,x028,x029,x030,x031,x032,x033,x034,x035,x036,x037,x038,x039,x040,x041,x042,x043,x044,x045,x046,x047,x048,x049,x050,x051,x052,x053,x054,x055,x056,x057,x058,x059,x060,x061,x062,x063,x064,x065,x066,x067,x068,x069,x070,x071,x072,x073,x074,x075,x076,x077,x078,x079,x080,x081,x082,x083,x084,x085,x086,x087,x088,x089,x090,x091,x092,x093,x094,x095,x096,x097,x098,x099,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x150,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163,x164,x165,x166,x167,x168,x169,x170,x171,x172,x173,x174,x175,x176,x177,x178,x179,x180,x181,x182,x183,x184,x185,x186,x187,x188,x189,x190,x191,x192,x193,x194,x195,x196,x197,x198,x199,x200,x201,x202,x203,x204,x205,x206,x207,x208,x209,x210,x211,x212,x213,x214,x215,x216,x217,x218,x219,x220,x221,x222,x223,x224,x225,x226,x227,x228,x229,x230,x231,x232,x233,x234,x235,x236,x237,x238,x239,x240,x241,x242,x243,x244,x245,x246,x247,x248,x249,x250,x251,x252,x253,x254,x255,x256,x257,x258,x259,x260,x261,x262,x263,x264,x265,x266,x267,x268,x269,x270,x271,x272,x273,x274,x275,x276,x277,x278,x279,x280,x281,x282,x283,x284,x285,x286,x287,x288,x289,x290,x291,x292,x293,x294,x295,x296,x297,x298,x299,x300,x301,x302,x303,x304,y
0,1084094,426.0,39.0,128.0,426.0,0,0,0,0,0,2,4,4,9,19,5,14,8,5,6,2,3,2,1,1,1,1,6,1,5,0,0,0,0,0,0,0,0,0,0,0.6959,197600,60073,140000.0,200.0,2,0,0,0,0,0,0,0,0,1,0,0.9754,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0000,1.0000,2,2,2,2,2,2,0,0,0,967,812.0,27600.0,51633,426.0,62.0,244.0000,0,0,NaN,0,1,1,1,1,1,1,9,2,0,1.0,0,61.0,53.0,57.0,483322,0.8827,1,1,1,0,0,NaN,NaN,NaN,NaN,0,0,0,0.1943,484289,0,NaN,0,0,0,484289,0,484289,1,1,1,0,0,1.0,NaN,NaN,NaN,0,346762,NaN,0,NaN,0,NaN,0,0,0,0,NaN,0,NaN,807
1,1287777,160.0,2.0,64.0,160.0,1,1,2,0,1,3,3,7,5,21,5,16,5,3,9,2,3,1,9,1,1,1,3,9,6,0,0,0,0,0,0,0,0,0,0,0.0269,25000,24327,18000.0,600.0,2,0,0,0,0,0,0,0,0,0,0,0.0557,0.0077,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0000,1.0000,2,2,2,2,1,0,0,0,0,673,557.0,18000.0,24327,160.0,102.0,131.0000,1,0,NaN,0,1,1,1,1,1,1,3,1,0,1.0,0,49.0,49.0,49.0,107316,0.9172,1,1,1,1,17318,9.0,2.0,5.5,0.8417,1,1,1,0.2356,125307,0,NaN,0,0,0,125307,0,125307,1,1,1,2,0,1.0,9.0,2.0,5.5,17318,124634,0.8417,0,NaN,17318,0.8417,1,1,1,0,NaN,0,NaN,819
2,1483016,163.0,16.0,104.0,239.0,0,0,0,1,0,1,0,0,6,8,3,5,4,2,7,3,3,2,0,0,0,1,0,0,4,1,0,0,0,1,1,0,0,0,1,0.0458,15400,14694,9000.0,500.0,1,1,0,0,0,0,0,0,0,0,0,0.1079,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.6666,0.6666,2,2,2,2,0,0,0,0,0,706,637.0,9000.0,14694,163.0,16.0,104.3333,0,0,NaN,0,1,1,1,1,0,0,0,0,0,NaN,0,NaN,NaN,NaN,0,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0.0308,706,0,NaN,0,0,0,706,0,706,1,1,1,0,0,NaN,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,0,0,0,0,NaN,0,NaN,803
3,959054,NaN,NaN,NaN,102.0,0,0,0,0,0,0,2,2,0,4,0,4,2,0,0,0,0,0,3,0,0,0,0,0,0,7,0,0,1,4,6,0,0,1,3,NaN,0

In [ ]:
# importing the test set
df_test = pd.read_csv("CreditScore_test.csv")
print(f"{df_test.shape[1]} columns, {df_test.shape[0]} rows")
# checking duplicated rows
print(f"{df_test.duplicated().sum()} duplicated rows")
df_test.head()

305 columns, 20000 rows
0 duplicated rows


,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x019,x020,x021,x022,x023,x024,x025,x026,x027,x028,x029,x030,x031,x032,x033,x034,x035,x036,x037,x038,x039,x040,x041,x042,x043,x044,x045,x046,x047,x048,x049,x050,x051,x052,x053,x054,x055,x056,x057,x058,x059,x060,x061,x062,x063,x064,x065,x066,x067,x068,x069,x070,x071,x072,x073,x074,x075,x076,x077,x078,x079,x080,x081,x082,x083,x084,x085,x086,x087,x088,x089,x090,x091,x092,x093,x094,x095,x096,x097,x098,x099,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x150,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163,x164,x165,x166,x167,x168,x169,x170,x171,x172,x173,x174,x175,x176,x177,x178,x179,x180,x181,x182,x183,x184,x185,x186,x187,x188,x189,x190,x191,x192,x193,x194,x195,x196,x197,x198,x199,x200,x201,x202,x203,x204,x205,x206,x207,x208,x209,x210,x211,x212,x213,x214,x215,x216,x217,x218,x219,x220,x221,x222,x223,x224,x225,x226,x227,x228,x229,x230,x231,x232,x233,x234,x235,x236,x237,x238,x239,x240,x241,x242,x243,x244,x245,x246,x247,x248,x249,x250,x251,x252,x253,x254,x255,x256,x257,x258,x259,x260,x261,x262,x263,x264,x265,x266,x267,x268,x269,x270,x271,x272,x273,x274,x275,x276,x277,x278,x279,x280,x281,x282,x283,x284,x285,x286,x287,x288,x289,x290,x291,x292,x293,x294,x295,x296,x297,x298,x299,x300,x301,x302,x303,x304,y
0,1566464,21.0,21.0,21.0,44.0,1,0,0,4,0,1,0,0,0,5,2,3,3,1,4,2,2,1,1,0,0,1,0,0,3,8,0,0,0,5,5,0,0,0,3,1.1716,600,-103,1000.0,200.0,0,0,0,0,0,0,0,0,0,0,4,1.1799,1.1633,2,0,0,1,2,2,2,0,0,0,0,0,0,0,0,0,2617,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,44.0,2,0,0,0,2,2,2,2,0,0,0,2,16,0,4,7,7,7,8,12,16,16,16,4,4,5,5,5,8,8,9,9,9,4,4,4,4,4,4,4,4,4,4,0,0,0,0,0,1,5.0,8,12,16,16,16,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,2,1,1,2,2,2,1,0,0,1,1,1,0,3,0,0,0,3,25,0,4,7,7,7,8,12,25,25,25,4,4,5,5,5,8,8,18,18,18,4,4,5,5,5,4,4,13,13,13,0,0,8,8,8,1.0000,1.0000,0,0,0,0,0,0,0,0,0,2145,1016.0,1000.0,-103,21.0,21.0,21.0,4,2,1.1716,703,1,1,1,1,0,0,0,0,0,NaN,0,NaN,NaN,NaN,0,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,1.0634,2145,2145,1.0634,2,2,2,703,703,0,1,1,1,0,0,5.0,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,0,0,1,1,5.0,0,NaN,406
1,925102,108.0,2.0,27.0,156.0,0,2,3,3,1,5,2,2,6,22,7,15,6,4,12,6,3,3,10,0,1,1,0,3,7,4,0,1,1,1,4,0,1,1,1,0.0786,58450,53854,30000.0,200.0,1,0,0,0,0,1,1,0,1,0,0,0.8612,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,1.0,0,0,0,0,0,2,0,0,0,2,2,1,0,0,0,1,1,0,1,1,0,0,0,6,0,0,0,1,5,0,0,0,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.6666,0.6666,5,3,2,2,2,2,1,2,3,4596,1744.0,30000.0,53854,108.0,2.0,26.0,3,0,NaN,0,1,1,1,1,0,0,0,0,0,NaN,0,NaN,NaN,NaN,0,NaN,0,0,0,1,13090,33.0,33.0,33.0,0.593,1,1,1,0.1228,17686,0,0.0000,0,0,0,17686,0,17686,1,1,1,1,0,2.0,33.0,33.0,33.0,13090,13090,0.593,0,NaN,13090,0.593,1,1,1,0,1.0,0,NaN,679
2,1273974,218.0,64.0,107.0,218.0,0,0,0,0,0,0,3,0,2,5,4,1,4,4,3,2,2,2,1,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0.1506,42900,36436,15000.0,2000.0,1,0,0,0,1,0,0,0,0,0,0,0.4788,0.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0000,1.0000,2,2,2,2,2,0,0,0,0,6464,6177.0,15000.0,21436,218.0,74.0,146.0,0,0,NaN,0,1,1,1,1,0,0,0,0,0,NaN,0,NaN,NaN,NaN,0,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN,0,0,0,0.3450,20153,0,NaN,0,0,0,20153,0,20153,1,1,1,1,0,1.0,74.0,74.0,74.0,13689,13689,1.014,0,NaN,13689,1.014,1,1,1,0,1.0,13689,1.014,750
3,1456723,NaN,NaN,NaN,146.0,1,0,0,1,1,8,0,0,1,11,0,11,6,0,1,0,1,0,10,0,1,1,0,1,1,19,1,

In [ ]:
# defining the target variable
target = 'y'

In [ ]:
# making a list of numerical variables
num_vars =  [var for var in df_train.columns if var != target]
print(f"Number of continuous variables: {len(num_vars)}")

Number of continuous variables: 304


In [ ]:
# making a list of the variables that contain missing values
vars_with_na = [
    var for var in df_train.columns if df_train[var].isnull().sum() > 0]

In [ ]:
# bringing from EDA the variables that contain missing values
# and influence the target
vars_with_na_ind = [
 'x002', 'x003', 'x004', 'x005', 'x041', 'x044', 'x045', 'x057', 'x058', 'x098',
 'x148', 'x155', 'x222', 'x223', 'x234', 'x235', 'x237', 'x238', 'x239', 'x242',
  'x253', 'x255', 'x256', 'x257', 'x259', 'x272', 'x275', 'x295']
print(f"Number of variables with na indication: {len(vars_with_na_ind)}")

Number of variables with na indication: 28


<b>Making missing values indicators</b>

In [ ]:
# replacing missing values with 1 and others with 0
for var in vars_with_na_ind:
    # add binary missing indicator (in train and test)
    df_train[var + '_na_ind'] = np.where(df_train[var].isnull(), 1, 0)
    df_test[var + '_na_ind'] = np.where(df_test[var].isnull(), 1, 0)

# adapting names of vars_with_na_ind it the list
vars_with_na_ind = [var + '_na_ind' for var in vars_with_na_ind]

In [ ]:
#  making a list of discrete variables
discrete_vars = [var for var in num_vars if len(df_train[var].unique()) < 30]
print(f"Number of discrete variables: {len(discrete_vars)}")

Number of discrete variables: 142


In [ ]:
# making a list of continuous variables
cont_vars =  [var for var in num_vars
              if var not in discrete_vars and var != target]
print(f"Number of continuous variables: {len(cont_vars)}")

Number of continuous variables: 162


<b>Imputing missing values</b>

In [ ]:
# replace missing values in discrete variables with mode
for var in discrete_vars:
    # calculating the mode using the train set
    mode_val = df_train[var].mode()[0]
    print(var, mode_val)
    # replacing missing values by the mode in train and test
    df_train[var].fillna(mode_val, inplace=True)
    df_test[var].fillna(mode_val, inplace=True)

x006 0
x010 0
x018 4
x019 0
x022 2
x023 0
x025 0
x026 1
x027 1
x032 0
x037 0
x038 0
x039 0
x040 0
x046 0
x047 0
x048 0
x049 0
x050 0
x051 0
x052 0
x053 0
x054 0
x055 0
x056 0
x060 0
x061 0
x062 0
x067 0
x068 0
x069 0
x070 0
x071 0
x076 0
x077 0
x078 0
x079 0
x080 0
x081 0
x082 0
x083 0
x084 0
x085 0
x086 0
x087 0
x088 0
x089 0
x090 0
x091 0
x092 0
x093 0
x094 0
x095 0
x096 0
x097 0
x099 0
x100 0
x101 0
x102 0
x103 0
x104 0
x105 0
x107 0
x108 0
x109 0
x110 0
x112 0
x113 0
x122 0
x123 0
x124 0
x132 0
x147 0
x148 1.0
x149 0
x150 0
x154 0
x155 1.0
x156 0
x157 0
x161 0
x162 1.0
x163 0
x164 0
x168 0
x169 0
x170 0
x171 0
x172 0
x173 0
x174 0
x175 0
x176 0
x177 0
x178 0
x179 0
x180 0
x182 0
x183 0
x184 0
x187 0
x197 0
x198 0
x207 0
x225 0
x226 0
x227 0
x228 0
x229 0
x230 0
x231 0
x241 0
x244 1
x245 1
x246 1
x247 1
x248 0
x249 0
x251 0
x252 0
x253 1.0
x254 0
x260 0
x261 0
x262 0
x263 0
x269 0
x270 0
x271 0
x276 0
x277 0
x278 0
x282 1
x283 1
x284 1
x286 0
x287 1.0
x298 1
x299 1
x300 1
x301 0
x30

In [ ]:
# replace missing values in continuous variables with mean
for var in cont_vars:
    # calculating the mean using the train set
    mean_val = df_train[var].mean()
    print(var, mean_val)
    # replacing missing values by the mean in train and test
    df_train[var].fillna(mean_val, inplace=True)
    df_test[var].fillna(mean_val, inplace=True)

x001 1218319.015
x002 125.60730717185386
x003 25.58252009870254
x004 65.37964628535953
x005 178.0435627487256
x007 0.69245
x008 1.385575
x009 1.1887375
x011 1.8390625
x012 1.6062625
x013 1.57845
x014 4.3162125
x015 12.941125
x016 4.084825
x017 8.8563
x020 6.8018
x021 2.281075
x024 4.7291625
x028 0.8006
x029 1.4785125
x030 4.0414
x031 5.3490625
x033 0.811075
x034 1.6966125
x035 3.1331375
x036 3.2245125
x041 0.47180038426494475
x042 19113.9435125
x043 11886.5168125
x044 23667.904228991236
x045 844.7447974224477
x057 0.6530358515231947
x058 0.27481076139919974
x059 2.3911125
x063 0.6880625
x064 1.5795375
x065 2.14535
x066 0.315975
x072 0.2794875
x073 1.3901875
x074 0.2479125
x075 2008.912275
x098 44.263467632412855
x106 0.560925
x111 2.87315
x114 0.5708375
x115 0.71855
x116 0.81755
x117 0.5495375
x118 1.0156625
x119 1.7501375
x120 2.3080625
x121 2.7604625
x125 0.2628875
x126 0.3158625
x127 0.35415
x128 0.664925
x129 1.1793
x130 1.5895125
x131 1.9429125
x133 0.1517625
x134 0.2697
x135 0.35

In [ ]:
# checking that train and test sets do not contain null values
[[var for var in num_vars if df_train[var].isnull().sum() > 0],
 [var for var in num_vars if df_test[var].isnull().sum() > 0]]

[[], []]

Discussion: target encoding of categorical variables is impossible because data set description lacks information on category variables.

<b>Storing engineered train and test sets</b>

In [ ]:
# making a list of engineered features
eng_feats = vars_with_na_ind + discrete_vars + cont_vars + [target]
print(f"Number of engineered features: {len(eng_feats)}")

Number of engineered features: 333


<h2>3. Feature Selection</h2><br>

<b>Removing discrete variables that don't vary with the target</b>

In [ ]:
# bringing from EDA the discrete variables that might influence the target
infl_discrete_vars = [
 'x010', 'x018', 'x019', 'x022', 'x023', 'x025', 'x026', 'x027', 'x039', 'x040',
 'x046', 'x047', 'x048', 'x049', 'x050', 'x051', 'x052', 'x053', 'x056', 'x060',
 'x061', 'x062', 'x068', 'x069', 'x070', 'x071', 'x076', 'x077', 'x078', 'x079',
 'x080', 'x081', 'x089', 'x093', 'x097', 'x099', 'x100', 'x101', 'x102', 'x103',
 'x104', 'x105', 'x107', 'x108', 'x109', 'x110', 'x112', 'x113', 'x122', 'x123',
 'x124', 'x132', 'x147', 'x148', 'x149', 'x150', 'x154', 'x155', 'x156', 'x157',
 'x161', 'x162', 'x163', 'x164', 'x168', 'x169', 'x170', 'x171', 'x172', 'x173',
 'x174', 'x175', 'x176', 'x177', 'x178', 'x179', 'x180', 'x183', 'x184', 'x187',
 'x197', 'x198', 'x207', 'x225', 'x226', 'x227', 'x228', 'x229', 'x230', 'x231', 
 'x241', 'x244', 'x245', 'x246', 'x247', 'x248', 'x249', 'x251', 'x252', 'x253',
 'x254', 'x260', 'x261', 'x262', 'x276', 'x277', 'x278', 'x282', 'x283', 'x284',
 'x286', 'x287', 'x301', 'x302']
print(len(infl_discrete_vars))

114


In [ ]:
# making a list of pre-selected features
pre_sel_feats = vars_with_na_ind + infl_discrete_vars + cont_vars + [target]
print(f"Number of pre-selected features: {len(pre_sel_feats)}")

Number of pre-selected features: 305


<b>Constant features</b>

In [ ]:
# to find variables that contain only 1 label/value
# nunique() method from pandas is used, which returns the number
# of different values in a variable.
constant_feats = [var for var in pre_sel_feats if df_train[var].nunique() == 1]

print(f"There is {len(constant_feats)} constant features")
constant_feats

There is 0 constant features


[]

<b>Quasi-constant features</b>

In [ ]:
# finding the features with low variance
sel = VarianceThreshold(threshold=0.01)
sel.fit(df_train)
quasi_constant = df_train.columns[~sel.get_support()]
print(f"There is {len(quasi_constant)} quasi-constant features")
quasi_constant

There is 9 quasi-constant features


Index(['x060', 'x067', 'x077', 'x083', 'x084', 'x093', 'x094', 'x095', 'x096'], dtype='object')

In [ ]:
# showing percentage of observations for each of the different values
# of the feature
for feat in quasi_constant:
    feat_values = df_train[feat].value_counts() / np.float(len(df_train))
    print(feat_values)

0    0.999975
1    0.000025
Name: x060, dtype: float64
0    1.0
Name: x067, dtype: float64
0    0.996613
1    0.003350
2    0.000037
Name: x077, dtype: float64
0    0.990663
1    0.009338
Name: x083, dtype: float64
0    0.99485
1    0.00515
Name: x084, dtype: float64
0    0.999275
1    0.000725
Name: x093, dtype: float64
0    1.0
Name: x094, dtype: float64
0    1.0
Name: x095, dtype: float64
0    1.0
Name: x096, dtype: float64


In [ ]:
# dropping the quasi-constant features from the train_set and test_set
df_train.drop(columns=quasi_constant,inplace=True)
df_test.drop(columns=quasi_constant,inplace=True)

In [ ]:
# check - finding the features with low variance
sel = VarianceThreshold(threshold=0.01)
sel.fit(df_train)
quasi_constant = df_train.columns[~sel.get_support()]
print(f"There is {len(quasi_constant)} quasi-constant features")
quasi_constant

There is 0 quasi-constant features


Index([], dtype='object')

In [ ]:
# showing length of the training dataset
df_train.shape

(80000, 324)

<b>Correlated features</b>

In [ ]:
# building a dataframe with the correlation between features
corrmat = df_train.corr()
corrmat = corrmat.abs().unstack()
corrmat = corrmat.sort_values(ascending=False)
corrmat = corrmat[corrmat >= 0.9]
corrmat = corrmat[corrmat < 1]
corrmat = pd.DataFrame(corrmat).reset_index()
corrmat.columns = ['feature1', 'feature2', 'corr']
print(len(corrmat), end='\n\n')
corrmat.head()

514



,feature1,feature2,corr
0,x004_na_ind,x002_na_ind,0.999852
1,x004_na_ind,x003_na_ind,0.999852
2,x002_na_ind,x004_na_ind,0.999852
3,x003_na_ind,x004_na_ind,0.999852
4,x222_na_ind,x223_na_ind,0.998233


In [ ]:
# finding groups of correlated features
grouped_feature_ls = []
correlated_groups = []
for feature in corrmat.feature1.unique():
    if feature not in grouped_feature_ls:
        # finding all features correlated to a single feature
        correlated_block = corrmat[corrmat.feature1 == feature]
        grouped_feature_ls = grouped_feature_ls + list(
            correlated_block.feature2.unique()) + [feature]
        # appending the block of features to the list
        correlated_groups.append(correlated_block)
print(f"found {len(correlated_groups)} correlated groups")
print(f"out of {df_train.shape[1]} total features")

found 59 correlated groups
out of 324 total features


In [ ]:
# printing out each group
for group in correlated_groups:
    print(group)
    print()

      feature1     feature2      corr
0  x004_na_ind  x002_na_ind  0.999852
1  x004_na_ind  x003_na_ind  0.999852

        feature1     feature2      corr
4    x222_na_ind  x223_na_ind  0.998233
22   x222_na_ind         x244  0.991505
24   x222_na_ind  x237_na_ind  0.991505
26   x222_na_ind  x239_na_ind  0.991505
27   x222_na_ind  x238_na_ind  0.991505
95   x222_na_ind  x058_na_ind  0.971973
96   x222_na_ind  x041_na_ind  0.971973
99   x222_na_ind  x057_na_ind  0.971973
482  x222_na_ind  x148_na_ind  0.904191

    feature1 feature2      corr
6       x279     x273  0.997320
102     x279     x292  0.971379
165     x279     x258  0.965053
178     x279     x281  0.963299

    feature1 feature2      corr
8       x111     x121  0.994722
195     x111     x120  0.961259
218     x111     x131  0.957741
375     x111     x141  0.927065
392     x111     x130  0.924919
496     x111     x146  0.902084

   feature1 feature2      corr
18     x222     x223  0.993121

    feature1     feature2      corr

In [ ]:
# making a function to delete less correlated features
# and leave one most correlated feature for each group
def less_correlated_features(df_group):
    # making unique list of correlated features
    correlated_feat = \
        df_group.feature2.append(df_group.feature1).unique().tolist()
    # adding target to the list
    correlated_feat.append(target)
    # making list of correlation coefficient with target
    corr_matrix = df_train[correlated_feat].corr(method='pearson')
    # making list of less correlatd features
    features_to_drop = pd.DataFrame(
        data=corr_matrix[target].drop(target)
            ).sort_values(by=target, ascending=False)
    features_to_drop = features_to_drop.iloc[1: ,:]
    features_to_drop = features_to_drop.index.tolist()
    
    return features_to_drop

In [ ]:
# deleting less correlated features
# and leaving one most correlated feature for each group
features_to_drop_total = []
for df_group in correlated_groups:
    feat = list(less_correlated_features(df_group))
    features_to_drop_total += feat 
print('Deleting {} features'.format(len(features_to_drop_total)))

# dropping the less correlated features from the train_set and test_set
df_train.drop(columns=features_to_drop_total, inplace=True)
df_test.drop(columns=features_to_drop_total, inplace=True)

Deleting 144 features


In [ ]:
# showing length of the training dataset
df_train.shape

(80000, 203)

<h2>4. Regression</h2>

In [ ]:
# making X_train, y_train, X_test and y_test
X_train = df_train.drop(columns=[target]).values
X_test = df_test.drop(columns=[target]).values
y_train = df_train[target].values
y_test = df_test[target].values

In [ ]:
# defining a regressor
regressor_xg = XGBRegressor(random_state=42)

# applying k-fold cross validation
rmse = cross_val_score(estimator=regressor_xg, X=X_train, y=y_train, cv=10)
print()
print('Cross validation')
print("rmse: {:.2f} %".format(rmse.mean()*100))
print("Standard Deviation: {:.2f} %".format(rmse.std()*100))

# fitting the regressor and predicting y_test
regressor_xg.fit(X_train, y_train)
y_pred = regressor_xg.predict(X_test)

# evaluation
y_train_hat = regressor_xg.predict(X_train)
print()
print('Evaluation')
print(f"RMSE train: {mean_squared_error(y_train, y_train_hat):.1f}")
print(f"RMSE test : {mean_squared_error(y_test, y_pred):.1f}")
print(f"r2 train  : {r2_score(y_train, y_train_hat):.4f}")
print(f"r2 test   : {r2_score(y_test, y_pred):.4f}")

[06:08:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:09:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:09:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:10:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:10:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:11:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:11:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:12:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

<h2>5. Discussion</h2>

With the selected features credit scores were predicted with RMSE_train of 1037.0 and RMSE_test of 1065.4. R2_train 0.9260 and R2_test 0.9246.